# Predicing pre- and postconditions using LLM #

## Imports ##

In [ ]:
# from runningLLAMA import llama_local_generate
from runningBaronLLM import baron_local_generate
import pandas as pd
import re
import time
import os

## Prepare the prompts ##
Multiple prompts for ablation study

In [9]:
# Prompt 1: Both vulnerability description AND CVSS vector
prompt_both = """
    Classify the vulnerability post-condition privilege as one of the following:
    - None: Attacker does not gain access to the system.  - User: Attacker gains
    user-level access (e.g., running code as a normal user, accessing user
    files).  - Root: Attacker gains full system or administrative access (e.g.,
    root privileges, complete control over the system or application).

    
    Vulnerability: {description}
    CVSS Vector: {cvss}
    
    Examples: Example 1: Vulnerability: XSS vulnerability allows stealing user
    session cookies.  CVSS Vector: CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:C/C:L/I:L/A:N
    Classification: User Justification: Attacker gains access to user session
    data but not system control.

    Example 2: Vulnerability: SQL injection allows database manipulation.  CVSS
    Vector: CVSS:3.1/AV:N/AC:L/PR:H/UI:N/S:U/C:H/I:H/A:H Classification: User
    Justification: Attacker gains database access but not full system control.

    Now classify the given vulnerability: Justification: [Your justification]
    ##POSTCONDITION [Your classification: None, User, or Root]
    """

# Prompt 2: Only vulnerability description
prompt_desc_only = """
    Classify the vulnerability post-condition privilege as one of the following:
    - None: Attacker does not gain access to the system.  - User: Attacker gains
    user-level access (e.g., running code as a normal user, accessing user
    files).  - Root: Attacker gains full system or administrative access (e.g.,
    root privileges, complete control over the system or application).

    
    Vulnerability: {description}
    
    Examples: Example 1: Vulnerability: XSS vulnerability allows stealing user
    session cookies.  
    Classification: User Justification: Attacker gains access to user session
    data but not system control.

    Example 2: Vulnerability: SQL injection allows database manipulation.  
    Classification: User Justification: Attacker gains database access but not full system control.

    Now classify the given vulnerability: Justification: [Your justification]
    ##POSTCONDITION [Your classification: None, User, or Root]
    """

# Prompt 3: Only CVSS vector
prompt_cvss_only = """
    Classify the vulnerability post-condition privilege as one of the following:
    - None: Attacker does not gain access to the system.  - User: Attacker gains
    user-level access (e.g., running code as a normal user, accessing user
    files).  - Root: Attacker gains full system or administrative access (e.g.,
    root privileges, complete control over the system or application).

    
    CVSS Vector: {cvss}
    
    Examples: Example 1: CVSS Vector: CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:C/C:L/I:L/A:N
    Classification: User Justification: Low impact on confidentiality and integrity suggests user-level access.

    Example 2: CVSS Vector: CVSS:3.1/AV:N/AC:L/PR:H/UI:N/S:U/C:H/I:H/A:H 
    Classification: User Justification: High impact but requires high privileges, suggesting user-level access.

    Now classify the given vulnerability: Justification: [Your justification]
    ##POSTCONDITION [Your classification: None, User, or Root]
    """


In [10]:
sys_prompt_both = """
    You are a cybersecurity vulnerability classification expert. Your task is to
    determine the post-condition privilege level after successful exploitation.

    POST-CONDITION PRIVILEGE LEVEL DEFINITIONS: - None: Attacker does not gain
    access to the system. No execution privileges are obtained.  - User:
    Attacker gains user-level access (e.g., running code as a normal user,
    accessing user files, limited privileges).  - Root: Attacker gains full
    system or administrative access (e.g., root privileges, complete control
    over the system or application, administrator rights).

    CLASSIFICATION INSTRUCTIONS: 1. Analyze both the CVE description and CVSS
    vector 2. Provide a brief justification 3. End your
    response with: ##POSTCONDITION [classification] 4. The classification must
    be EXACTLY one of: None, User, Root
    """

sys_prompt_desc_only = """
    You are a cybersecurity vulnerability classification expert. Your task is to
    determine the post-condition privilege level after successful exploitation.

    POST-CONDITION PRIVILEGE LEVEL DEFINITIONS: - None: Attacker does not gain
    access to the system. No execution privileges are obtained.  - User:
    Attacker gains user-level access (e.g., running code as a normal user,
    accessing user files, limited privileges).  - Root: Attacker gains full
    system or administrative access (e.g., root privileges, complete control
    over the system or application, administrator rights).

    CLASSIFICATION INSTRUCTIONS: 1. Analyze the CVE description 2. Provide a brief 
    justification 3. End your response with: ##POSTCONDITION [classification] 
    4. The classification must be EXACTLY one of: None, User, Root
    """

sys_prompt_cvss_only = """
    You are a cybersecurity vulnerability classification expert. Your task is to
    determine the post-condition privilege level after successful exploitation.

    POST-CONDITION PRIVILEGE LEVEL DEFINITIONS: - None: Attacker does not gain
    access to the system. No execution privileges are obtained.  - User:
    Attacker gains user-level access (e.g., running code as a normal user,
    accessing user files, limited privileges).  - Root: Attacker gains full
    system or administrative access (e.g., root privileges, complete control
    over the system or application, administrator rights).

    CLASSIFICATION INSTRUCTIONS: 1. Analyze the CVSS vector 2. Provide a brief 
    justification 3. End your response with: ##POSTCONDITION [classification] 
    4. The classification must be EXACTLY one of: None, User, Root
    """

In [ ]:
# Recommended params for BaronLLM
temperature = 0.3
top_p = 0.9     
seed = 42
# Small number of output tokens to speed
max_tokens = 256   

## Format output ##

In [12]:
def format_post_condition(text):
    """
    Extract post-condition privilege classification from LLM response text.
    Returns the last valid classification found and whether extraction was successful.
    """
    # Define the regex pattern for matching privilege classification
    privilege_pattern = r'^(None|User|Root)[.:\s]*$'
    
    # Split the text into lines (from bottom up) and search for a matching line
    lines = text.strip().splitlines()
    for line in reversed(lines):
        line = line.strip()
        if re.match(privilege_pattern, line):
            # Extract just the classification word
            match = re.match(r'^(None|User|Root)', line)
            if match:
                return match.group(1), True
    
    # If no exact match found, look for the classification word anywhere in the text
    text_reversed = text[::-1]
    for word in ["tooR", "resU", "enoN"]:  # Reversed words
        if word in text_reversed:
            pos = text_reversed.find(word)
            return word[::-1], True  # Reverse back to original
    
    # If still not found, return the entire text and mark as failed
    return text, False

<!--  -->

In [14]:
data_set_file_path = "../datasets/postcondition-dataset-finished.tsv"
# run_evaluation(data_set_file_path)


### Ablation Study ###

In [ ]:
def run_ablation_study(file_path):
    """
    Run ablation study with three different prompt variations:
    1. Both vulnerability description AND CVSS vector
    2. Only vulnerability description  
    3. Only CVSS vector
    """
    
    # Define the three prompt variations with their corresponding system prompts
    prompts = {
        "both": (sys_prompt_both, prompt_both),
        "desc_only": (sys_prompt_desc_only, prompt_desc_only), 
        "cvss_only": (sys_prompt_cvss_only, prompt_cvss_only)
    }
    
    for prompt_name, (system_prompt, prompt_template) in prompts.items():
        print(f"\n=== Running ablation study with prompt: {prompt_name} ===")
        
        start_time = time.time()
        count_chars = 0
        instructions_failed = 0
        
        data = pd.read_csv(file_path, encoding='utf-8', sep='\t')
        all_results = []
        all_full_responses = []
        
        for index, row in data.iterrows():
            # Format the prompt based on the current variation
            if prompt_name == "both":
                llm_prompt = prompt_template.format(description=row['DESCRIPTION'], cvss=row['CVSS'])
            elif prompt_name == "desc_only":
                llm_prompt = prompt_template.format(description=row['DESCRIPTION'])
            elif prompt_name == "cvss_only":
                llm_prompt = prompt_template.format(cvss=row['CVSS'])
            
            try:
                # Get prediction from the model using the specific system prompt for this variation
                output = baron_local_generate(system_prompt, llm_prompt, max_tokens=max_tokens, temperature=temperature, top_p=top_p, seed=seed)
                count_chars += len(output)
                
                # Store the full response
                all_full_responses.append(f"=== CVE {index+1} ===\n{output}\n")
                
                # Try to extract post-condition from the response
                answer, success = format_post_condition(output)
                if not success:
                    instructions_failed += 1
                
                all_results.append(answer)
                print(f"{index+1} {answer}")
            except Exception as e:
                answer = 'Error'
                error_msg = f"=== CVE {index+1} ===\nERROR: {str(e)}\n"
                all_full_responses.append(error_msg)
                all_results.append(answer)
                print(f'Exception at row {index+1}')
                print(e)
        
        time_taken = time.time() - start_time
        print(f'Time taken for {prompt_name}: {time_taken}')
        print(f'#Characters generated: {count_chars}')
        print(f'#Instructions failed: {instructions_failed}')
        
        # Create output directory for ablation study
        output_dir = os.path.join('responses', 'ablation-study')
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Save results with prompt variation in filename
        base_filename = os.path.basename(file_path).split('.')[0]
        out_result = os.path.join(output_dir, f'SENG402_{base_filename}_postcondition_{prompt_name}.txt')
        with open(out_result, 'w', encoding='utf-8') as f:
            f.write('\n'.join(all_results))
        
        out_full_responses = os.path.join(output_dir, f'SENG402_{base_filename}_full_postcondition_responses_{prompt_name}.txt')
        with open(out_full_responses, 'w', encoding='utf-8') as f:
            f.write('\n'.join(all_full_responses))
        
        print(f'Results saved to: {out_result}')
        print(f'Full responses saved to: {out_full_responses}')
        print(f'------- {prompt_name} Done --------\n')

# Run the ablation study
data_set_file_path = "../datasets/postcondition-dataset-finished.tsv"
run_ablation_study(data_set_file_path)


=== Running ablation study with prompt: both ===


## Helper functions for evaluation ##

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate_predictions(true_labels_file, pred_labels_file, prompt_name):
    """
    Calculate F1 score and accuracy for predictions.
    
    Args:
        true_labels_file: Path to file with ground truth labels (one per line)
        pred_labels_file: Path to file with predicted labels (one per line)
        prompt_name: Name of the prompt variant for reporting
    
    Returns:
        Dictionary with F1 and accuracy scores
    """
    # Read true labels and predicted labels
    with open(true_labels_file, 'r', encoding='utf-8') as f:
        true_labels = [line.strip() for line in f.readlines()]
    
    with open(pred_labels_file, 'r', encoding='utf-8') as f:
        pred_labels = [line.strip() for line in f.readlines()]
    
    # Ensure same length
    min_len = min(len(true_labels), len(pred_labels))
    true_labels = true_labels[:min_len]
    pred_labels = pred_labels[:min_len]
    
    # Handle any prediction errors (replace with most common class or 'User')
    valid_classes = ['None', 'User', 'Root']
    pred_labels_clean = []
    failed_count = 0
    
    for pred in pred_labels:
        if pred in valid_classes:
            pred_labels_clean.append(pred)
        else:
            pred_labels_clean.append('User')  # Default fallback
            failed_count += 1
    
    # Calculate metrics
    accuracy = accuracy_score(true_labels, pred_labels_clean)
    f1_macro = f1_score(true_labels, pred_labels_clean, average='macro')  # Treats all classes equally
    f1_weighted = f1_score(true_labels, pred_labels_clean, average='weighted')  # Weighted by class frequency
    
    return {
        'prompt_name': prompt_name,
        'accuracy': accuracy,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'failed_extractions': failed_count,
        'total_predictions': len(pred_labels)
    }



In [ ]:
def compare_ablation_with_ground_truth(dataset_file="../../datasets/postcondition-dataset-finished.tsv", 
                                     results_dir='../responses/ablation-study'):
    """
    Compare all three ablation study results with ground truth from the dataset.
    """
    from sklearn.metrics import accuracy_score, f1_score, classification_report
    
    # Read ground truth from dataset
    dataset = pd.read_csv(dataset_file, sep='\t', encoding='utf-8', keep_default_na=False, na_values=[''])
    ground_truth = dataset['POSTCONDITION'].str.strip().tolist()
    
    prompt_variants = ['both', 'desc_only', 'cvss_only']
    results_summary = []
    
    print("=== ABLATION STUDY vs GROUND TRUTH COMPARISON ===\n")
    print("Variant\t\tAccuracy\tF1-Macro\tFailed Extractions")
    print("-" * 55)
    
    best_accuracy = 0
    best_f1_macro = 0
    best_variant_acc = ""
    best_variant_f1_macro = ""
    
    for variant in prompt_variants:
        pred_file = os.path.join(results_dir, f'SENG402_postcondition-dataset-finished_postcondition_{variant}.txt')
        
        if os.path.exists(pred_file):
            # Read predictions
            with open(pred_file, 'r', encoding='utf-8') as f:
                predictions = [line.strip() for line in f.readlines()]
            
            # Ensure same length
            min_len = min(len(ground_truth), len(predictions))
            gt_subset = ground_truth[:min_len]
            pred_subset = predictions[:min_len]
            
            # Handle failed extractions (replace invalid predictions with most common class)
            valid_classes = ['None', 'User', 'Root']
            pred_clean = []
            failed_count = 0
            
            for pred in pred_subset:
                if pred in valid_classes:
                    pred_clean.append(pred)
                else:
                    pred_clean.append('User')  # Default fallback
                    failed_count += 1
            
            # Calculate metrics
            accuracy = accuracy_score(gt_subset, pred_clean)
            f1_macro = f1_score(gt_subset, pred_clean, average='macro')
            
            # Store results
            results_summary.append({
                'variant': variant,
                'accuracy': accuracy,
                'f1_macro': f1_macro,
                'failed_extractions': failed_count,
                'total_predictions': len(pred_subset)
            })
            
            print(f"{variant:12}\t{accuracy:.3f}\t\t{f1_macro:.3f}\t\t{failed_count}/{len(pred_subset)}")
            
            # Track best performance
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_variant_acc = variant
            if f1_macro > best_f1_macro:
                best_f1_macro = f1_macro
                best_variant_f1_macro = variant
                
        else:
            print(f"{variant:12}\tFile not found")
    
    print("\n=== BEST PERFORMANCE ===")
    print(f"Best Accuracy: {best_variant_acc} ({best_accuracy:.3f})")
    print(f"Best F1-Macro: {best_variant_f1_macro} ({best_f1_macro:.3f})")
    
    # Detailed per-class analysis for best performing variant
    print(f"\n=== DETAILED ANALYSIS FOR BEST VARIANT ({best_variant_f1_macro}) ===")
    best_pred_file = os.path.join(results_dir, f'SENG402_postcondition-dataset-finished_postcondition_{best_variant_f1_macro}.txt')
    
    if os.path.exists(best_pred_file):
        with open(best_pred_file, 'r', encoding='utf-8') as f:
            best_predictions = [line.strip() for line in f.readlines()]
        
        min_len = min(len(ground_truth), len(best_predictions))
        gt_subset = ground_truth[:min_len]
        pred_subset = best_predictions[:min_len]
        
        # Clean predictions
        pred_clean = [pred if pred in valid_classes else 'User' for pred in pred_subset]
        
        print(classification_report(gt_subset, pred_clean, target_names=['None', 'User', 'Root']))
        
        # Show some examples of correct and incorrect predictions
        print("\n=== SAMPLE PREDICTIONS ===")
        for i, (true_label, pred_label) in enumerate(zip(gt_subset, pred_clean)):
            status = "✓" if true_label == pred_label else "✗"
            print(f"Row {i+1}: {status} True: {true_label:4} | Pred: {pred_label:4}")
            if i >= 9:  # Show first 10 examples
                break
    
    return results_summary

# Run the comparison
results = compare_ablation_with_ground_truth()

=== ABLATION STUDY vs GROUND TRUTH COMPARISON ===

Variant		Accuracy	F1-Macro	Failed Extractions
-------------------------------------------------------
both        	0.727		0.719		0/33
desc_only   	0.576		0.534		0/33
cvss_only   	0.303		0.252		0/33

=== BEST PERFORMANCE ===
Best Accuracy: both (0.727)
Best F1-Macro: both (0.719)

=== DETAILED ANALYSIS FOR BEST VARIANT (both) ===
              precision    recall  f1-score   support

        None       1.00      0.45      0.62        11
        User       0.90      0.75      0.82        12
        Root       0.56      1.00      0.71        10

    accuracy                           0.73        33
   macro avg       0.82      0.73      0.72        33
weighted avg       0.83      0.73      0.72        33


=== SAMPLE PREDICTIONS ===
Row 1: ✗ True: Root | Pred: User
Row 2: ✓ True: Root | Pred: Root
Row 3: ✗ True: None | Pred: User
Row 4: ✓ True: None | Pred: None
Row 5: ✗ True: None | Pred: Root
Row 6: ✓ True: User | Pred: User
Row 7: ✓ Tr